In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [3]:
#https://www.w3schools.com/sql/default.asp
#https://www.w3resource.com/sql-exercises/movie-database-exercise/joins-exercises-on-movie-database.php
con = sqlite3.connect('Db-IMDB.db')

### Assignment
1.List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [22]:
qn1 = pd.read_sql_query('''select Name,title,year from Person p join M_Director md on p.PID = md.PID join Movie m on m.MID = md.MID where m.MID in
                            (select MID from Movie where (year%4=0 and year%100!=0) and (year%4=0 or year%100=0 and year%400=0) and MID in
                            (select MID from M_Genre where GID in(select GID from Genre where Name LIKE '%comedy%')))''', con)
print(qn1.head(10))

              Name                              title  year
0     Milap Zaveri                         Mastizaade  2016
1     Milap Zaveri                         Mastizaade  2016
2     Danny Leiner  Harold & Kumar Go to White Castle  2004
3     Danny Leiner  Harold & Kumar Go to White Castle  2004
4   Anurag Kashyap                 Gangs of Wasseypur  2012
5   Anurag Kashyap                 Gangs of Wasseypur  2012
6     Frank Coraci        Around the World in 80 Days  2004
7     Frank Coraci        Around the World in 80 Days  2004
8    Griffin Dunne             The Accidental Husband  2008
9    Griffin Dunne             The Accidental Husband  2008


2. List the names of all the actors who played in the movie 'Anand' (1971)

In [23]:
qn2 = pd.read_sql_query("""WITH MOVIE_ANAND AS
                        (SELECT M.title Movie, MC.PID
                        FROM MOVIE M
                        JOIN M_CAST MC ON M.MID = MC.MID
                        WHERE M.title = 'Anand')

                        SELECT DISTINCT TRIM(P.NAME) Actors
                        FROM MOVIE_ANAND MA
                        JOIN PERSON P ON TRIM(P.PID) = TRIM(MA.PID""", con)

In [24]:
qn2

,Actors
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo
5,Asit Kumar Sen
6,Dev Kishan
7,Atam Prakash
8,Lalita Kumari
9,Savita


3) List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [25]:
qn3=pd.read_sql_query("""WITH Actors_Before_1970 AS
                        (SELECT DISTINCT TRIM(MC.PID) PID
                        FROM MOVIE M
                        JOIN M_CAST MC ON M.MID = MC.MID
                        WHERE CAST(SUBSTR(M.year,-4) AS UNASSIGNED) < 1970),

                        Actors_After_1990 AS
                        (SELECT DISTINCT TRIM(MC.PID) PID
                        FROM MOVIE M
                        JOIN M_CAST MC ON M.MID = MC.MID
                        WHERE CAST(SUBSTR(M.year,-4) AS UNASSIGNED) > 1990),

                        ACTORS AS
                        (SELECT A_1970.PID PID
                        FROM Actors_Before_1970 A_1970
                        JOIN Actors_After_1990 A_1990 ON A_1970.PID = A_1990.PID)

                        SELECT DISTINCT TRIM(P.NAME) Actors_Before_1970_After_1990
                        FROM PERSON P
                        JOIN ACTORS A ON A.PID = TRIM(P.PID)""", con)

In [26]:
qn3

,Actors_Before_1970_After_1990
0,Rishi Kapoor
1,Amitabh Bachchan
2,Asrani
3,Zohra Sehgal
4,Parikshat Sahni
5,Rakesh Sharma
6,Sanjay Dutt
7,Ric Young
8,Yusuf
9,Suhasini Mulay


4. List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [27]:
qn4=pd.read_sql_query("""SELECT DISTINCT TRIM(P.Name) Directors, COUNT(DISTINCT MD.MID) Movie_Count
                        FROM PERSON P
                        JOIN M_DIRECTOR MD ON TRIM(P.PID) = TRIM(MD.PID)                                                    
                        GROUP BY Directors
                        HAVING Movie_Count >= 10
                        ORDER BY Movie_Count DESC""", con)

In [28]:
qn4

,Directors,Movie_Count
0,David Dhawan,39
1,Mahesh Bhatt,36
2,Priyadarshan,30
3,Ram Gopal Varma,30
4,Vikram Bhatt,29
5,Hrishikesh Mukherjee,27
6,Yash Chopra,21
7,Basu Chatterjee,19
8,Shakti Samanta,19
9,Subhash Ghai,18


5.a) For each year, count the number of movies in that year that had only female actors.

In [29]:
qn5a = pd.read_sql_query("""WITH MOVIE_MALE_NONE AS
                            (SELECT MC.MID MID_F
                            FROM M_CAST MC
                            JOIN PERSON P ON TRIM(P.PID) = TRIM(MC.PID)
                            WHERE TRIM(P.GENDER) IN ('Male', 'None'))

                            SELECT CAST(SUBSTR(M.year,-4) AS UNASSIGNED) Year, COUNT(DISTINCT TRIM(MID)) Female_Movie_Count
                            FROM MOVIE M
                            WHERE TRIM(MID) NOT IN (SELECT MID_F FROM MOVIE_MALE_NONE)
                            GROUP BY CAST(SUBSTR(M.year,-4) AS UNASSIGNED)
                            ORDER BY Year""", con)

In [30]:
qn5a

,Year,Female_Movie_Count
0,1939,1
1,1999,1
2,2000,1
3,2009,1
4,2012,1
5,2018,2


5. b) Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [31]:
qn5b = pd.read_sql_query("""WITH Movie_Non_Females AS
                            (SELECT DISTINCT TRIM(MC.MID) MID
                            FROM M_CAST MC
                            JOIN PERSON P ON TRIM(MC.PID) = TRIM(P.PID)
                            WHERE TRIM(P.GENDER) IN ('Male','None')),

                            MOVIE_FEMALE_Year AS
                            (SELECT CAST(SUBSTR(M.YEAR,-4) AS UNASSIGNED) YEAR,
                            COUNT(DISTINCT TRIM(MID)) Female_Movie_Only
                            FROM Movie M
                            WHERE TRIM(MID) NOT IN (SELECT MID FROM Movie_Non_Females)
                            GROUP BY CAST(SUBSTR(M.year,-4) AS UNASSIGNED)),

                            MOVIES_YEAR AS
                            (SELECT CAST(SUBSTR(M.YEAR,-4) AS UNASSIGNED) Year,
                            COUNT(DISTINCT TRIM(MID)) Total_Movies
                            FROM MOVIE M
                            GROUP BY CAST(SUBSTR(M.YEAR,-4) AS UNASSIGNED))

                            SELECT MY.YEAR, MY.Total_Movies,
                            ROUND((IFNULL(MF.Female_Movie_Only, 0) * 100)/MY.Total_Movies, 2) Female_Movie_Percentage
                            FROM MOVIES_YEAR MY
                            LEFT OUTER JOIN MOVIE_FEMALE_Year MF ON
                            TRIM(MY.YEAR) = TRIM(MF.YEAR)
                            ORDER BY Female_Movie_Percentage DESC""", con)

In [32]:
qn5b

,Year,Total_Movies,Female_Movie_Percentage
0,1939,2,50.0
1,1999,66,1.0
2,2000,64,1.0
3,2018,104,1.0
4,1931,1,0.0
5,1936,3,0.0
6,1941,1,0.0
7,1943,1,0.0
8,1946,2,0.0
9,1947,2,0.0


6) Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [38]:
qn6 = pd.read_sql_query("""WITH CAST AS
                            (SELECT COUNT(DISTINCT TRIM(MC.PID)) Cast_Count, MC.MID MID
                            FROM M_CAST MC
                            GROUP BY MC.MID)

                            SELECT M.title Movie, C.Cast_Count
                            FROM MOVIE M
                            JOIN CAST C ON C.MID = M.MID
                            ORDER BY Cast_Count DESC
                            LIMIT 1""", con)

In [34]:
qn6

,Movie,Cast_Count
0,Ocean's Eight,238


7) A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [35]:
qn7=pd.read_sql_query("""WITH  YEARS_UNIQUE AS
                       (SELECT DISTINCT
                        CAST(SUBSTR(year,-4) AS UNSIGNED) YEAR,
                        CAST(SUBSTR(year,-4) AS UNSIGNED) DECADE_START,
                        CAST(SUBSTR(year,-4) AS UNSIGNED) + 9 DECADE_END,
                        'Decade of : ' || SUBSTR(year,-4) DECADE
                        FROM MOVIE),

                        MOVIE_COUNT_YEARS AS
                       (SELECT COUNT(DISTINCT MID) Movie_Count, CAST(SUBSTR(year,-4) AS UNSIGNED) Year
                        FROM MOVIE
                        GROUP BY CAST(SUBSTR(year,-4) AS UNSIGNED)),

                        MOVIE_COUNT_DECADE AS 
                       (SELECT SUM(Movie_Count) Total_Movies, YU.DECADE
                        FROM MOVIE_COUNT_YEARS MCY, YEARS_UNIQUE YU
                        WHERE MCY.YEAR BETWEEN YU.DECADE_START AND YU.DECADE_END
                        GROUP BY YU.DECADE)

                        SELECT Decade, Total_Movies
                        FROM MOVIE_COUNT_DECADE
                        WHERE Total_Movies = (SELECT MAX(Total_Movies)
                        FROM MOVIE_COUNT_DECADE)""", con)

In [36]:
qn7

,DECADE,Total_Movies
0,Decade of : 2008,1205


8) Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies).

In [39]:
qn8= pd.read_sql_query("""WITH ACTORS_MOVIE_YEAR AS
                        (SELECT TRIM(MC.PID) PID, CAST(SUBSTR(year,-4) AS UNASSIGNED) Year,
                        COUNT(DISTINCT TRIM(M.MID)) Number_of_Mov
                        FROM M_CAST MC, MOVIE M
                        WHERE TRIM(MC.MID) = TRIM(M.MID) 
                        GROUP BY TRIM(MC.PID), CAST(SUBSTR(year,-4) AS UNASSIGNED)
                        ORDER BY NUMBER_OF_MOV DESC),
    
                        ACTORS_MORE_THAN_YEAR AS
                        (SELECT AMY.PID, COUNT(AMY.YEAR) AS Number_of_Years, MIN(AMY.YEAR) AS Min_Year,
                        MAX(AMY.YEAR) AS Max_Year
                        FROM ACTORS_MOVIE_YEAR AMY
                        GROUP BY AMY.PID
                        HAVING COUNT(AMY.YEAR) > 1),
    
                        ACTORS_NUMBER_MORE_THAN_YEAR AS
                        (SELECT AMY.PID, AMY.YEAR, AMY.YEAR+4 AS Year_4, AMY.NUMBER_OF_MOV,
                        ATY.MIN_YEAR, ATY.MAX_YEAR
                        FROM ACTORS_MOVIE_YEAR AMY, ACTORS_MORE_THAN_YEAR ATY
                        WHERE AMY.PID = ATY.PID),
    
                        NUMBER_MOVIE_PRESENT AS
                        (SELECT AM.PID, NY.YEAR, SUM(AM.NUMBER_OF_MOV) AS NUMBER_OF_MOVIE_PRESENT
                        FROM ACTORS_NUMBER_MORE_THAN_YEAR AM, ACTORS_NUMBER_MORE_THAN_YEAR NY
                        WHERE AM.PID = NY.PID AND
                        AM.YEAR BETWEEN NY.MIN_YEAR AND NY.YEAR
                        GROUP BY AM.PID, NY.YEAR),
    
                        ACTOR_MOVIE_4_YEAR AS
                        (SELECT AM.PID, NY.YEAR, SUM(AM.NUMBER_OF_MOV) AS ACTOR_MOVIE_4_YEARS_PRESENT
                        FROM ACTORS_NUMBER_MORE_THAN_YEAR AM, ACTORS_NUMBER_MORE_THAN_YEAR NY
                        WHERE AM.PID = NY.PID AND
                        AM.YEAR BETWEEN NY.MIN_YEAR AND NY.YEAR_4 AND
                        NY.YEAR_4 <= NY.MAX_YEAR
                        GROUP BY AM.PID, NY.YEAR)
    
                        SELECT DISTINCT TRIM(P.NAME) AS ACTORS_NEVER_UNEMPLOYED_MORE_THAN_3_YEARS
                        FROM PERSON P
                        WHERE TRIM(P.PID) NOT IN (SELECT DISTINCT NMP.PID
                        FROM NUMBER_MOVIE_PRESENT NMP, ACTOR_MOVIE_4_YEAR AM_4
                        WHERE NMP.PID = AM_4.PID AND
                        NMP.YEAR = AM_4.YEAR AND
                        NMP.NUMBER_OF_MOVIE_PRESENT = AM_4.ACTOR_MOVIE_4_YEARS_PRESENT) """, con)

In [40]:
qn8

,ACTORS_NEVER_UNEMPLOYED_MORE_THAN_3_YEARS
0,Christian Bale
1,Cate Blanchett
2,Benedict Cumberbatch
3,Naomie Harris
4,Andy Serkis
5,Peter Mullan
6,Jack Reynor
7,Eddie Marsan
8,Tom Hollander
9,Matthew Rhys


9) Find all the actors that made more movies with Yash Chopra than any other director.


In [6]:
qn9= pd.read_sql_query("""select Director, Actor, Count(*) as Movies_with_YashChopra 
from(select p1.Name as Director, m1.title as Movie from Person p1 Inner Join M_Director
md on TRIM(md.PID)=p1.PID Inner Join Movie m1 on TRIM(md.MID)=m1.MID and 
p1.Name LIKE 'Yash%' Group By p1.Name, m1.title) t1 Inner Join
(select p2.Name as Actor,m2.title as Movie from Person p2 Inner Join 
M_Cast mc on TRIM(mc.PID)=p2.PID Inner Join Movie m2 on TRIM(mc.MID)=m2.MID 
Group By p2.Name, m2.title) t2 on t1.Movie=t2.Movie Group By t1.Director, t2.Actor having Movies_with_YashChopra >1
Order By Movies_with_YashChopra DESC""", con)

In [7]:
qn9

,Director,Actor,Movies_with_YashChopra
0,Yash Chopra,Jagdish Raj,11
1,Yash Chopra,Manmohan Krishna,10
2,Yash Chopra,Manmohan Krishna,10
3,Yash Chopra,Iftekhar,9
4,Yash Chopra,Madan Puri,8
5,Yash Chopra,Vikas Anand,8
6,Yash Chopra,Anupam Kher,7
7,Yash Chopra,Shashi Kapoor,7
8,Yash Chopra,Anupam Kher,7
9,Yash Chopra,Shashi Kapoor,7


10) The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.



In [43]:
qn10 = pd.read_sql_query("""WITH SHAHRUK_0 AS
                        (SELECT TRIM(P.PID) PID
                        FROM PERSON P
                        WHERE TRIM(P.NAME) like '%Shahrukh%'),
                        
                        SHAHRUK_1_MOV AS
                        (SELECT DISTINCT TRIM(MC.MID) MID, S_0.PID
                        FROM SHAHRUK_0 S_0, M_CAST MC
                        WHERE TRIM(MC.PID) = S_0.PID),
                        
                        SHAHRUK_1_ACTS AS
                        (SELECT DISTINCT TRIM(MC.PID) PID
                        FROM M_CAST MC, SHAHRUK_1_MOV SM_1
                        WHERE TRIM(MC.MID) = SM_1.MID AND
                        TRIM(MC.PID) <> SM_1.PID),
                        
                        SHAHRUK_2_MOV AS
                        (SELECT DISTINCT TRIM(MC.MID) MID, SA_1.PID
                        FROM SHAHRUK_1_ACTS SA_1, M_CAST MC
                        WHERE TRIM(MC.PID) = SA_1.PID)
                        
                        SELECT DISTINCT TRIM(P.NAME) Shahruk_2_Actors
                        FROM PERSON P, M_CAST MC, SHAHRUK_2_MOV SM_2
                        WHERE TRIM(MC.PID) = TRIM(P.PID) AND
                        MC.MID = SM_2.MID AND
                        MC.PID <> SM_2.PID""", con)

In [44]:
qn10

,Shahruk_2_Actors
0,Freida Pinto
1,Caroline Christl Long
2,Rajeev Pahuja
3,Michelle Santiago
4,Jandre le Roux
5,Raj Awasti
6,Michael Chapman
7,James Heron
8,Alex Jaep
9,Marian Lorencik
